In [8]:
import torch
from torch import nn
from torchinfo import summary
import numpy as np

In [2]:
cfgs =  {
            "A" : [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
            "B" : [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
            "D" : [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
            "E" : [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"] 
        }

In [11]:
class VggNet(nn.Module):
    def __init__(self, cfg, num_classes=1000):
        super().__init__()

        self.features = self.make_layers(cfg)
        self.AdaptAvgPool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
                                        nn.Linear(512*7*7, 4096),
                                        nn.ReLU(),
                                        nn.Linear(4096, 4096),
                                        nn.ReLU(),
                                        nn.Linear(4096, num_classes))
        
        # weight, bias initialization
        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                nn.init.kaiming_normal_(module.weight)
                nn.init.constant_(module.bias, 0.0)
            elif isinstance(module, nn.Linear):
                nn.init.normal_(module.weight, mean=0, std=0.01)
                nn.init.constant_(module.bias, 0.0)
    
    def forward(self, x):
        x = self.features(x)
        x = self.AdaptAvgPool(x)
        x = torch.flatten(x, start_dim=1)
        x = self.classifier(x)
        return x
    
    def make_layers(self, cfg):
        layers = []
        curr_channel = 3

        for cmd in cfg:
            if type(cmd) == int:
                layers += [ nn.Conv2d(curr_channel, cmd, 3, padding=1),
                            nn.BatchNorm2d(cmd),
                            nn.ReLU()]
                curr_channel = cmd
            else:
                layers += [nn.MaxPool2d(2, 2)]
        
        return nn.Sequential(*layers)

In [12]:
model = VggNet(cfgs['A'], num_classes=1000)
summary(model, (64, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
VggNet                                   [64, 1000]                --
├─Sequential: 1-1                        [64, 512, 1, 1]           --
│    └─Conv2d: 2-1                       [64, 64, 32, 32]          1,792
│    └─BatchNorm2d: 2-2                  [64, 64, 32, 32]          128
│    └─ReLU: 2-3                         [64, 64, 32, 32]          --
│    └─MaxPool2d: 2-4                    [64, 64, 16, 16]          --
│    └─Conv2d: 2-5                       [64, 128, 16, 16]         73,856
│    └─BatchNorm2d: 2-6                  [64, 128, 16, 16]         256
│    └─ReLU: 2-7                         [64, 128, 16, 16]         --
│    └─MaxPool2d: 2-8                    [64, 128, 8, 8]           --
│    └─Conv2d: 2-9                       [64, 256, 8, 8]           295,168
│    └─BatchNorm2d: 2-10                 [64, 256, 8, 8]           512
│    └─ReLU: 2-11                        [64, 256, 8, 8]           --
